In [185]:
import modules.singleton_reduction as singleton
import modules.isolate_reduction as isolate
from tqdm import tqdm
import importlib
importlib.reload(singleton)
importlib.reload(isolate)
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from concurrent.futures import ThreadPoolExecutor
import io
import copy


In [186]:
# Load the full graph
edges = pd.read_csv("../data/edges_updated_reversed.csv", sep=' ')
edges_array = np.array(edges.loc[:, ['Source', 'Target']])
nodes = pd.read_csv(r"../data/nodes.csv", sep=",")
nodes_array = nodes["# index"]

G_full = nx.DiGraph()
G_full.add_edges_from(edges_array)
G_full.add_nodes_from(nodes_array)

In [187]:
importlib.reload(singleton)
importlib.reload(isolate)

G = copy.deepcopy(G_full)
G = singleton.singleton_reduction(G)
G_full_sr = copy.deepcopy(G)
# isolatedNodes = list(nx.isolates(G))
G, isolatedNum = isolate.isolate_reduction(G)

Node count before singleton reduction:  58742
Nodes before:  52124
Nodes after:  19257


In [104]:
import modules.sir_threaded as sir
import importlib
import copy
importlib.reload(sir)
init_infected = [9]
max_steps = 100
G_temp = copy.deepcopy(G)

# Uncomment below block if you want to only view nodes that are accessible from init_infected
G_temp = G_temp.subgraph(sir.get_accessible_sus_nodes(G_temp, init_infected)).copy()

In [ ]:
# Uncomment below block if you want to only view nodes that are accessible from init_infected
# print("Nodes to calculate: ", len(G_temp))
# pos = nx.spring_layout(G_temp, seed=8020, gravity=0.75)

In [105]:
importlib.reload(sir)

_, _, _, _, infotext_total, constants_total = sir.sir_model(G_temp, G_full_sr, init_infected=init_infected, infection_rate=0.2, recovery_rate=0.1, max_steps=max_steps, doSingletonReduction=True, noticeability_rates=(0.0001,0.01), network_type='full')

[DEBUG2] False
Running model...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Step:  1
[DEBUG] QT:  0
[INFO] Infecting singletons and neighbors + Recovering infected nodes...
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG

In [60]:
infotext_total

[{'ssw': 950,
  'sso': 5668,
  'it': 1,
  'st': 4990,
  'rt': 0,
  'qt': 0,
  'ss': 6618,
  'is': 0,
  'rs': 0},
 {'ssw': 801,
  'sso': 5668,
  'it': 617,
  'st': 4374,
  'rt': 0,
  'qt': 0,
  'ss': 6469,
  'is': 149,
  'rs': 0},
 {'ssw': 667,
  'sso': 5668,
  'it': 1148,
  'st': 3843,
  'rt': 0,
  'qt': 0,
  'ss': 6335,
  'is': 283,
  'rs': 0},
 {'ssw': 576,
  'sso': 5668,
  'it': 1580,
  'st': 3411,
  'rt': 0,
  'qt': 0,
  'ss': 6244,
  'is': 374,
  'rs': 0},
 {'ssw': 493,
  'sso': 5668,
  'it': 1955,
  'st': 3036,
  'rt': 0,
  'qt': 0,
  'ss': 6161,
  'is': 457,
  'rs': 0},
 {'ssw': 423,
  'sso': 5668,
  'it': 2300,
  'st': 2691,
  'rt': 0,
  'qt': 0,
  'ss': 6091,
  'is': 527,
  'rs': 0},
 {'ssw': 359,
  'sso': 5668,
  'it': 2607,
  'st': 2384,
  'rt': 0,
  'qt': 0,
  'ss': 6027,
  'is': 591,
  'rs': 0},
 {'ssw': 315,
  'sso': 5668,
  'it': 2895,
  'st': 2096,
  'rt': 0,
  'qt': 0,
  'ss': 5983,
  'is': 635,
  'rs': 0},
 {'ssw': 273,
  'sso': 5668,
  'it': 3102,
  'st': 1889,
  'rt

In [58]:
constants_total

{'tn': 58742, 'xt': 53751, 'virusFoundStep': 61}

In [108]:
# Graphical Analysis of a single run
plt.close()

# create data
x = range(len(infotext_total))
it_total = [i['it'] for i in infotext_total]
st_total = [i['st'] for i in infotext_total]
rt_total = [i['rt'] for i in infotext_total]
qt_total = [i['qt'] for i in infotext_total]

loss_total = [i['loss']['q']+i['loss']['i'] for i in infotext_total]
max_loss_total = max(loss_total)
loss_total = [i/max_loss_total*(constants_total['tn']-constants_total['xt']) for i in loss_total]


# plot lines
plt.plot(x, st_total, label = "Total susceptible", alpha=0.5, color='g')
plt.plot(x, it_total, label = "Total infected", alpha=0.5, color='r')
plt.plot(x, rt_total, label = "Total recovered", alpha=0.5, color='b')
plt.plot(x, qt_total, label = "Total quarantined", alpha=0.5, color='0.7')
plt.plot(x, loss_total, label = "Loss (scaled)", alpha=0.5, color='k')
plt.vlines(constants_total['virusFoundStep'], 0, constants_total['tn']-constants_total['xt'], color='k', linestyle='dotted', lw=1)
plt.legend()
plt.savefig(f"results/test.png", format="PNG", bbox_inches='tight')

In [95]:
# Loss analysis of a single run
plt.close()
loss = {'quarantine': 1, 'infected': 1}

x = range(len(infotext_total))
it_total = [i['it'] for i in infotext_total]
st_total = [i['st'] for i in infotext_total]
rt_total = [i['rt'] for i in infotext_total]
qt_total = [i['qt'] for i in infotext_total]

plt.stackplot(x, (
                    [i*loss['infected'] for i in it_total],
                    [i*loss['quarantine'] for i in qt_total]
                ),
             labels=['Loss from infected', 'Loss from quarantines'],
             colors=['#ff0000','#444444'],
              alpha=0.8)
plt.legend()
plt.savefig(f"results/test2.png", format="PNG", bbox_inches='tight')

In [190]:
importlib.reload(sir)
import os
from multiprocessing import Pool
from functools import partial

# Loss analysis of multiple runs
run_count = 10000

init_infected = [9]
# init_infected = 
max_steps = 1000
infection_rate = 0.2
recovery_rate = 0.1
noticeability_rates = (0.0001,0.01)
quarantine_length = 5
contact_tracing = 3
loss_matrix = ((1000, 0.5),(0.2, 0.1))

loss_results = None

tempList = []
for i in range(run_count):
    tempList.append(G)

worker = partial(
    sir.simulate_threaded,
    G=G, 
    G_full_sr=G_full_sr, 
    init_infected=init_infected, 
    infection_rate=infection_rate, 
    recovery_rate=recovery_rate, 
    max_steps=max_steps, 
    noticeability_rates=noticeability_rates, 
    quarantine_length=quarantine_length,
    contact_tracing=contact_tracing,
    loss_matrix=loss_matrix,
    tempList=tempList,
)

with Pool(16) as p:
    loss_results = p.map(worker, range(run_count))

total_loss_results = [x['q'] + x['i'] for x in loss_results] 

(np.mean(total_loss_results), np.std(total_loss_results), np.mean([x['q'] for x in loss_results]), np.mean([x['i'] for x in loss_results] ))

Run:  0  --- Steps taken:  91  --- Loss:  4211807.099999997  --- Q_Loss:  876.1  --- I_Loss:  4210930.999999997
Copy Time:  0.04569411277770996  --- Simulation Time:  0.17345476150512695
Run:  1  --- Steps taken:  88  --- Loss:  3758838.000000002  --- Q_Loss:  814.0000000000005  --- I_Loss:  3758024.000000002
Copy Time:  0.026538848876953125  --- Simulation Time:  0.36372923851013184
Run:  2  --- Steps taken:  99  --- Loss:  3249340.300000006  --- Q_Loss:  460.9000000000001  --- I_Loss:  3248879.400000006
Copy Time:  0.03568005561828613  --- Simulation Time:  0.16490697860717773
Run:  157  --- Steps taken:  136  --- Loss:  4323524.800000008  --- Q_Loss:  834.4000000000005  --- I_Loss:  4322690.400000008
Copy Time:  0.08129692077636719  --- Simulation Time:  0.24762797355651855
Run:  158  --- Steps taken:  105  --- Loss:  3395474.600000001  --- Q_Loss:  663.0000000000006  --- I_Loss:  3394811.600000001
Copy Time:  0.03142237663269043  --- Simulation Time:  0.2005901336669922
Run:  3  --

(3931041.866410002, 536668.091925645, 937.6647100000001, 3930104.2017000015)

In [193]:
plt.cla()
plt.hist(total_loss_results, bins=200)
plt.savefig("results/test3.png")

# loss_matrix = ((1, 0),(0.2, 0.1)) w/ 100 runs
## max_steps = 100
quarantine_length 5: 15342.928
quarantine_length 5, ct level 2: 14599.878000000002
quarantine_length 10: 15261.373
quarantine_length 5, ct level 1, nt_rate * 10 noticed: 14576.289000000004
quarantine_length inf, ct level 1: 20178.598

## max_steps = 1000 (*)
quarantine_length 5, ct level 1: 14514.140000000009
quarantine_length 10, ct level 1: 15413.901000000007
quarantine_length 5, ct level 2: 14708.250000000007
quarantine_length 5, ct level 3: 15418.860000000004
quarantine_length 2, ct level 1: 15211.508000000003
no quarantines: 14869.428000000004

# loss_matrix = ((1, 0),(0.2, 0.1)) w/ 1000 runs
## init_infected = [9] ; max_steps = 1000 ; infection_rate = 0.2 ; recovery_rate = 0.1 ; noticeability_rates = (0.0001,0.01)
no quarantining: ~14470 loss and ~3637 std
quarantine_length 5, ct level 0: ~14492 loss and ~3610 std
quarantine_length 10, ct level 0: ~14804 loss and ~3727 std
quarantine_length 5, ct level 1: ~14948 loss and ~3747 std
quarantine_length 10, ct level 1: ~15314 loss and ~3720 std
quarantine_length 5, ct level 2: ~14725 loss and ~3844 std
quarantine_length 10, ct level 2: ~15281 loss and ~4066 std
quarantine_length 5, ct level 3: ~14989 loss and ~3863 std
quarantine_length 10, ct level 3: ~15204 loss and ~3892 std

## init_infected = [9] ; max_steps = 1000 ; infection_rate = 0.1 ; recovery_rate = 0.1 ; noticeability_rates = (0.0001,0.01) 
no quarantining: ~12231 loss and ~3892 std
quarantine_length 5, ct level 0: ~12197 loss and ~3997 std
quarantine_length 10, ct level 0: ~12284 loss and ~3977 std
quarantine_length 5, ct level 1: ~12591 loss and ~4007 std (q: 437 loss -- i: 11927)
quarantine_length 10, ct level 1: ~12502 loss and ~3913 std (q: 706 loss -- i: 11796)
quarantine_length 5, ct level 2: ~12287 loss and ~3809 std (q: 411 loss -- i: 11876)
quarantine_length 10, ct level 2: ~12673 loss and ~4074 std (q: 725 loss -- i: 11948)

# loss_matrix = ((1000, 0.5),(0.2, 0.1)) w/ 1000 runs
## init_infected = [9] ; max_steps = 1000 ; infection_rate = 0.2 ; recovery_rate = 0.1 ; noticeability_rates = (0.0001,0.01)
no quarantining: ~3976276 loss and ~524603 std
quarantine_length 5, ct level 0: ~3942146 loss and ~547844 std (q: 350 -- i: 3941796)
quarantine_length 10, ct level 0: ~3966693 loss and ~531257 std (q: 490 -- i: 3966203)
quarantine_length 5, ct level 1: ~3920625 loss and ~538986 std (q: 986 -- i: 3919639)
quarantine_length 10, ct level 1: ~3918447 loss and ~530026 std (q: 1320 -- i: 3917127)
quarantine_length 5, ct level 2: ~3881514 loss and ~542880 std (q: 947 -- i: 3880568)
quarantine_length 10, ct level 2: ~3932767 loss and ~533736 std (q: 1298 -- i: 3931469)
quarantine_length 5, ct level 3: ~3973245 loss and ~502952 std (q: 938 -- i: 3972307)

quarantine_length 10, ct level 3: ~3942986 loss and ~531066 std (q: 949 -- i: 3942037)
